In [397]:
import pandas as pd
import numpy as np
import numpy.random as nr
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm
import pickle
model = pickle.load(open('regmodel.sav', 'rb'))

In [398]:
df = pd.read_csv('Test_Data_Prepped.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
CustomerID              500 non-null int64
CountryRegionName       500 non-null object
Education               500 non-null object
Occupation              500 non-null object
Gender                  500 non-null object
MaritalStatus           500 non-null object
NumberCarsOwned         500 non-null int64
NumberChildrenAtHome    500 non-null int64
TotalChildren           500 non-null int64
YearlyIncome            500 non-null int64
Age                     500 non-null int64
ChildrenOut             500 non-null int64
dtypes: int64(7), object(5)
memory usage: 47.0+ KB


In [399]:
def encode_string(cat_features):
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

In [400]:
categorical_columns = ['Education', 'Occupation', 'Gender', 'MaritalStatus']
Features = encode_string(df['CountryRegionName'])
for col in categorical_columns:
    temp = encode_string(df[col])
    Features = np.concatenate([Features, temp], axis = 1)

print(Features.shape)
print(Features[:2, :])

(500, 20)
[[0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0.]]


In [401]:
Features = np.concatenate([Features, np.array(df[['NumberCarsOwned', 
                            'NumberChildrenAtHome', 'YearlyIncome', 'Age', 'ChildrenOut']])], axis = 1)
print(Features.shape)
print(Features[:2, :])

(500, 25)
[[0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00
  1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00
  0.00000e+00 1.00000e+00 2.00000e+00 0.00000e+00 8.69310e+04 5.20000e+01
  5.00000e+00]
 [0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00 1.00000e+00
  1.00000e+00 0.00000e+00 2.00000e+00 2.00000e+00 1.00125e+05 3.30000e+01
  2.00000e+00]]


In [402]:
scaler = preprocessing.StandardScaler().fit(Features[:,22:])
Features[:,22:] = scaler.transform(Features[:,22:])

In [403]:
regPred = pd.DataFrame(model.predict(Features))

In [404]:
regPred.head()

,0
0,42.041016
1,106.250000
2,48.054688
3,87.773438
4,60.156250


In [405]:
regPred.rename(columns = {0: 'AveMonthSpend'}, inplace=True)
regPred['CustomerID'] = df['CustomerID']
regPred = regPred[['CustomerID', 'AveMonthSpend']]
regPred.to_csv('regPred.csv', mode='w', index=False)

In [406]:
regPred.head()

,CustomerID,AveMonthSpend
0,18988,42.041016
1,29135,106.250000
2,12156,48.054688
3,13749,87.773438
4,27780,60.156250
